# 本地环境加载

In [5]:
import os
from dotenv import load_dotenv
from langchain_qwq import ChatQwen
from langchain.agents import create_agent

load_dotenv()
qwen_api_key = os.getenv("DASHSCOPE_API_KEY")

model = ChatQwen(
    model="qwen-flash",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
)

# 静态模型
静态模型在创建代理时进行一次配置，并在整个执行过程中保持不变。这是最常见且最直接的方法。

In [ ]:
model = ChatQwen(
    model="qwen-flash",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
)
agent = create_agent(model)

ai_msg = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "你好，你是什么模型"
        }
    ]
})

print(ai_msg["messages"][-1].content)
print(ai_msg["messages"][-1].response_metadata["model_name"])
# print(ai_msg["messages"][-1])

# 动态模型
动态模型的选择是在运行时环境基于当前状态以及上下文信息。这使得复杂的路由逻辑和成本优化成为可能。

需要使用@wrap_model_call装饰器创建中间件，该装饰器会在请求中修改模型

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatQwen(model="qwen-flash", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", )
advanced_model = ChatQwen(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", )


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """根据对话的复杂程度选择合适的模型。"""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # 对于较长的对话，请使用高级模型。
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))


agent = create_agent(
    model=basic_model,  # Default model
    middleware=[dynamic_model_selection]
)


ai_msg = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "今有雉（鸡）、兔同笼，上有三十五头（总头数35），下有九十四足（总脚数94）。问雉、兔各几何？让我们一步一步思考"
        }
    ]
})

print(ai_msg["messages"][-1].content)
print(ai_msg["messages"][-1].response_metadata["model_name"])

# 工具 & System
1. 多个工具按顺序调用（由单个提示触发）
2. 在适当的情况下并行调用工具
3. 基于之前结果的动态工具选择
4. 工具重试逻辑和错误处理
5. 工具调用间的状态持久化

In [ ]:
from langchain.tools import tool
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@tool
def search(query: str) -> str:
    """调用搜索工具"""
    return f"You searched for: {query}"

@tool
def get_weather(location: str) -> str:
    """调用天气工具"""
    return f"The weather in {location} is sunny, 72℃"

@wrap_tool_call
def handle_tool_errors(request, handler):
    """使用自定义消息处理工具执行错误。"""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model,
    tools=[search, get_weather],
    middleware=[handle_tool_errors],
    system_prompt="你是一位得力的助手。请务必言简意赅、准确无误。"

)

from langchain.messages import SystemMessage, HumanMessage

agent = create_agent(
    model,
    tools=[], #如果提供的工具列表为空，则代理将由一个没有工具调用功能的 LLM 节点组成。
    middleware=[handle_tool_errors],
    system_prompt=SystemMessage(
        content=[
            {
                 "type": "text",
                "text": "You are an AI assistant tasked with analyzing literary works.",
            },{
                "type": "text",
                "text": "<the entire contents of 'Pride and Prejudice'>",
                "cache_control": {"type": "ephemeral"} #该cache_control字段{"type": "ephemeral"}告诉 model 缓存该内容块，从而降低使用同一系统提示符的重复请求的延迟和成本。
            }
        ]
    )
)



{'messages': [HumanMessage(content='确定目前最流行的无线耳机并确认其供货情况。', additional_kwargs={}, response_metadata={}, id='987030fa-4c4e-49c2-9f20-ff9d071458eb'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 209, 'total_tokens': 235, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen-flash', 'system_fingerprint': None, 'id': 'chatcmpl-06d0d740-2573-9415-8559-47b08b536c62', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b925f-7dd2-7bb3-89f3-5bea467a9ac9-0', tool_calls=[{'name': 'search', 'args': {'query': 'currently popular wireless earbuds and their availability'}, 'id': 'call_e77cc88c5bdd492a82d448', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 209, 'output_tokens': 26, 'total_tokens': 235, 'input_token_details': {'cache_read': 0}, 'output_token_details': {